In [ ]:
"""
Folding by FFT and smoothing with Savizky-Golay (this last, just for poor statistics spectra). Fitting by lmfit library.
Likewise, it is included an option to identify phases by K-Nearest Neighbors (KNN) from hyperfine parameters results comparison with a local database
The next lines prepare Drive connection and import necessary libraries
"""
!pip install lmfit
from google.colab import drive
drive.mount('/content/drive/', force_remount= True)
img = '/content/drive/MyDrive/Colab\ Notebooks/PyMossFit/My File'

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lmfit import Parameters, minimize, fit_report, Model
from scipy.optimize import curve_fit
from scipy.constants import *
from scipy.integrate import trapezoid
from scipy.signal import savgol_filter
from scipy.stats import norm
from pathlib import Path

path= Path(img); file= path.stem; title= path.parent.name; full= path.parents[0]; print(file, title, full)

In [ ]:
"""
Loud and folding raw datafile subsection
"""
y= np.loadtxt(img, skiprows=0); #y=y[:,1] #add delivery=" " or delimiter=" " in load txt if your file includes 2 columns. Instead, comments y=y[:,1]
N=len(y); N2=int(N/2); N4=int(N/4)

fecha = str(input('Ingrese la fecha de calibración (AAAAMMDD): '),)
vel = float(input('Ingrese el rango de V (en mm/s): ' ), )
np.savetxt(f"{full}/{file}-calib.txt", (fecha,vel, N), fmt='%s')

#y= savgol_filter(y, 5, 2) #Just if the spectrum has poor statistic. Instead, comments it
plt.plot(y)
plt.show()

"""
Folding by FFT (based on Nyquist-Shannon sampling theorem, https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter24.02-Discrete-Fourier-Transform.html)
"""
f= abs(np.fft.fft(y)); Nf=pd.Series(f[N2-5:N2+5]).idxmax()+(N2-5)
plt.plot(f[1:N])
plt.show()

for i in range(0, Nf-1):
    y[[i]]=y[[i]]+y[[N-1-i]]

y=y[0:N2-1]

"""
Normalization
"""
y2= np.concatenate([y[2:30],y[N2-30:N2-2]])

ymax=np.mean(y2)
ymax=int(ymax)

for i in range(0, N2-1):
    y[[i]]=y[[i]]/ymax

x=np.arange(1,N2)

"""
From channels to mm/s
"""
v=np.loadtxt(f"{full}/{file}-calib.txt"); vmax=v[[1]]

x=np.linspace(start = 0, stop = N2-1, num= N2)

for i in range(0, N2-1):
    x[[i]]=(i-N4)*vmax/N4

x= x[0:N2-1]

print('Mean Background:', ymax); print('Folding channel:', Nf)

plt.plot(x,y, marker="o")
plt.show()

np.savetxt(f"{full}/{file}.csv", list(zip(x,y)), delimiter=",", fmt='%1.6f')

In [ ]:
"""
Reading folded datafile
"""
col_list=[0, 1]

x, y= np.loadtxt(f"{full}/{file}.csv", delimiter=",", usecols=col_list, unpack=True)

"""
Functions definition for fitting, based in lorentzian shapes
"""
# Mössbauer constants for doublet (transition level factors)
positions_d = np.array([-1 , 1]) / 2  # Relatives to quadrupolar splitting
intensities_d = np.array([1, 1])  # Relatives to line intensities

# Mössbauer constants for sextets (transition level factors)
positions_x = np.array([-3., -1.7, -0.5, 0.5, 1.7, 3.]) / 2  # Relatives to hyperfine Field
intensities_x = np.array([1, 2, 1., 1., 2, 1])  # Relatives to line intensities

def lorentzian(x, amplitude, center, width):
    """Singlet Lorentzian Function"""
    return amplitude * (2*width / ((np.pi) * ((x - center)**2 + width**2)))

def doublet_lorentzian(x, delta, quad, gamma, scale):
    """
    Doublet model for Mössbauer spectrum of quadrupolar splitting.

    Parameters:
    - delta: Isomer Shift (IS)
    - quad: Quadrupolar Splitting (QS)
    - gamma: Lorentzian function linewidth
    - scale: intensity scale factor
    """
    y = np.zeros_like(x)
    for i, (pos, inten) in enumerate(zip(positions_d, intensities_d)):
        center = delta + pos * quad
        y += lorentzian(x, scale * inten, center, gamma)
    return y

def sextet_lorentzian(x, delta, B_hf, gamma, scale):
    """
    Sextet model for Mössbauer spectrum of Zeeman splitting..

    Parámetros:
    - delta: Isomer shift (IS)
    - B_hf: hyperfine magnetic field
    - gamma: Lorentzian function linewidth
    - scale: intensity scale factor
    """
    y = np.zeros_like(x)
    for i, (pos, inten) in enumerate(zip(positions_x, intensities_x)):
        center = delta + pos * B_hf
        y += lorentzian(x, scale * inten, center, gamma)
    return y

# Lmfit for model definition and fitting
from lmfit import Model, minimize, fit_report
doublet1 = Model(doublet_lorentzian, prefix= 'd1_')
doublet2 = Model(doublet_lorentzian, prefix= 'd2_')
combined_model= doublet1 + doublet2

# Configuration of initial parameters and their limits
params = combined_model.make_params(d1_delta=1.1, d1_quad=2.9, d1_gamma=0.35, d1_scale=0.2, d2_delta=0.4, d2_quad=0.4, d2_gamma=0.35, d2_scale=0.2)
params['d1_delta'].set(min=-0.7, max=1.5)       # Ejemplo de límite para delta
params['d1_quad'].set(min=0, max=3.5)        # Límite para desdoblamiento cuadrupolar
params['d1_gamma'].set(min=0.0, max=0.45)      # Límite para ancho de línea
params['d1_scale'].set(min=0.0, max=1)      # Límite para escala de intensidad

params['d2_delta'].set(min=-0.7, max=1.5)       # Ejemplo de límite para delta
params['d2_quad'].set(min=0, max=0.95)        # Límite para desdoblamiento cuadrupolar
params['d2_gamma'].set(min=0.0, max=0.35)      # Límite para ancho de línea
params['d2_scale'].set(min=0.0, max=1)      # Límite para escala de intensidad

def linear_fitting_lmfit(params, x, y):
    y_fit= 1-combined_model.eval(params=params, x=x)

    return y_fit-y

# Results extraction and their saving in a DataFrame
param_headers = []
param_values = []
param_errors = []

# Fitting with a linear regression model
result = minimize(linear_fitting_lmfit, params, args=(x,y), method='least_squares')
best_fit= 1-combined_model.eval(params=result.params, x=x)

doublet1_fit= doublet1.eval(params=result.params, x=x)
doublet2_fit= doublet2.eval(params=result.params, x=x)

for name, param in result.params.items():
      param_headers.append(name)
      param_values.append(param.value)
      param_errors.append(param.stderr)

#Subespectral area are calculated as: i= trapz(1-z, x); i1= trapz(1-z1, x)/i*100; i2=trapz(1-z2, x)/i*100; etc con z=z1+z2+...
i = trapezoid(doublet1_fit + doublet2_fit, x)
i1 = trapezoid(doublet1_fit, x)/i*100
i2 = trapezoid(doublet2_fit, x)/i*100

e= (y-best_fit)/y*100

# Results printing
print(fit_report(result))

"""
PLOTS OF ESPECTRA AND SUBESPECTRA
"""
plt.style.use('bmh')

fig, (ax1, ax2) = plt.subplots(2, sharex=True, height_ratios=[1,3.5]); fig.suptitle(f"{title}" " - " f"{file}")
ax1.scatter (x, e, c= 'black', marker= '+')
ax1.set_ylim(-1.5,1.5)
ax1.set_ylabel('Error (%)'); #plt.axis('tight')

ax2.scatter (x, y, c= 'black', marker= '+')
ax2.set_xlabel('V(mm/s)')
ax2.set_ylabel('Relative Transmission(a.u.)')
ax2.plot(x, best_fit, c='red')
ax2.plot(x, 1-doublet1_fit, c='blue', label= 'QS 1')
ax2.plot(x, 1-doublet2_fit, c='green', label= 'QS 2')
ax2.legend(handlelength=4, loc='lower left', shadow=True)

plt.show()

"""
OUTPUT DATAFILE GENERATION: DATA Y PARAMETERS
"""
np.savetxt(f"{full}/{file}-plot.csv", list(zip(x, y, doublet1_fit, doublet2_fit, best_fit)), delimiter=',', fmt='%1.6e') #

# DataFrame Creation
results_df = pd.DataFrame({
    'Parameter': param_headers,
    'Value': param_values,
    'Error': param_errors
})

df = pd.DataFrame({
    'Ancho(mm/s)': results_df.loc[results_df['Parameter'].isin(['d1_gamma', 'd2_gamma']), 'Value'].values.tolist(),
    'IS (mm/s)': results_df.loc[results_df['Parameter'].isin(['d1_delta', 'd2_delta']), 'Value'].values.tolist(),
    'Quad Splitting (mm/s)': results_df.loc[results_df['Parameter'].isin(['d1_quad', 'd2_quad']), 'Value'].values.tolist(),
    'Bhf (T)': [0, 0], 'Áreas (%)': [i1, i2] # Ensure 'Bhf (T)' has the same length as others
})

#df= df.apply(lambda x: round (x,1))
df.to_csv(f"{full}/{file}-report.csv", index=False)

In [ ]:
"""
KNN algorithm for potential phases identification
"""
from sklearn.neighbors import NearestNeighbors

# 1. Google Drive Mounting
from google.colab import drive
drive.mount('/content/drive')

# 2. Louding reference data  (Database)
reference_path = '/content/drive/MyDrive/Colab\ Notebooks/PyMossFit/reference_data.csv'  # Reference data path
df_ref = pd.read_csv(reference_path)

# Function for mean range definition (i.e..: "0.37-0.45" → 0.41)
def parse_value(value):
    if isinstance(value, str) and '-' in value:
        min_val, max_val = map(float, value.split('-'))
        return (min_val + max_val) / 2
    return float(value)

# Relevant columns processing
cols = ['IS (mm/s)', 'Quad Splitting (mm/s)', 'Bhf (T)']
for col in cols:
    df_ref[col] = df_ref[col].apply(parse_value)

# 3. Louding experimental data
experimental_path = f"{full}/{file}-report.csv"  # ¡Ajusta la ruta!
df_exp = pd.read_csv(experimental_path)

# 4. Pre-processing of experimental data (management of NaN)
X_exp = df_exp[cols].fillna(0).values  # Si Bhf no existe, reemplazar NaN por 0

# 5. KNN Model Training
X_ref = df_ref[cols].values
model = NearestNeighbors(n_neighbors=3, metric='euclidean')
model.fit(X_ref)

# 6. Matching coincidences
distances, indices = model.kneighbors(X_exp)

# 7. Showing results
for i, (dist, idx) in enumerate(zip(distances, indices)):
    print(f"\nMuestra experimental {i+1}:")
    for j, (d, pos) in enumerate(zip(dist, idx)):
        compound = df_ref.iloc[pos]['Compound Name']
        formula = df_ref.iloc[pos]['Chemical Formula']
        is_ref = df_ref.iloc[pos]['IS (mm/s)']
        qs_ref = df_ref.iloc[pos]['Quad Splitting (mm/s)']
        bhf_ref = df_ref.iloc[pos]['Bhf (T)']
        print(f"  Match {j+1}: {compound} ({formula})")
        print(f"    IS: {is_ref:.2f} mm/s | QS: {qs_ref:.2f} mm/s | Bhf: {bhf_ref:.1f} T")
        print(f"    Distancia euclidiana: {d:.2f}\n")

print("## Use this result in an orientative way. It is recomended to complement it with compositional and structural  information of the samples ##")